## The P0 Compiler

### The P0 Grammar

    selector ::= { "[" expression "]" | "." ident}
    factor ::= ident selector | integer | "(" expression ")" | "{" [expression {"," expression}] "}" | ("¬" | "#" | "∁") factor
    term ::= factor {("×" | "div" | "mod" | "∩" | "and") factor}
    simpleExpression ::= ["+" | "-"] term {("+" | "-" | "∪" | "or") term}
    expression ::= simpleExpression
        {("=" | "≠" | "<" | "≤" | ">" | "≥" | "∈" | "⊆" | "⊇" | "↦") simpleExpression}
    statementList ::= statement {";" statement}
    statementBlock ::= statementList {statementList}
    statementSuite ::= statementList | INDENT statementBlock DEDENT
    statement ::=
        ident selector ":=" expression |
        ident {"," ident} (":=" expression {"," expression} |
            "←" ident "(" [expression {"," expression}] ")") |
        "if" expression "then" statementSuite ["else" statementSuite] |
        "while" expression "do" statementSuite
        "for" type "do" statementSuite
    type ::=
        ident |
        "[" expression ".." expression "]" "→" type |
        "[" expression {"," expression} "]" "→" type |
        "(" typedIds ")" |
        "set" "[" expression ".." expression "]" |
        ident "in" type
    typedIds ::= ident {"," ident} ":" type {"," ident {"," ident} ":" type}.
    declarations ::= 
        {"const" ident "=" expression}
        {"type" ident "=" type}
        {"var" typedIds}
        {"procedure" ident "(" [typedIds] ")" [ "→" "(" typedIds ")" ] body}
    body ::= INDENT declarations (statementBlock | INDENT statementBlock DEDENT) DEDENT
    program ::= declarations "program" ident body

In [ ]:
import nbimporter; nbimporter.options["only_defs"] = False
import copy

import SC  #  used for SC.init, SC.sym, SC.val, SC.error
from SC import IDENT, NUMBER, TIMES, DIV, MOD, PLUS, MINUS, AND, OR, \
    EQ, NE, LT, GT, LE, GE, SEMICOLON, COMMA, COLON, BECOMES, PERIOD, DOTDOT, \
    NOT, LPAREN, RPAREN, LBRAK, RBRAK, LBRACE, RBRACE, LARROW, RARROW, CARD, \
    COMPLEMENT, UNION, INTERSECTION, ELEMENT, SUBSET, SUPERSET, IF, THEN, ELSE, \
    WHILE, DO, CONST, TYPE, VAR, SET, PROCEDURE, PROGRAM, INDENT, DEDENT, EOF, \
    ARROW, FOR, IN, getSym, mark

import ST  #  used for ST.init
from ST import Var, LoopVar, Ref, Const, Type, Proc, StdProc, Int, Bool, \
    Record, Array, PartialArray, Set, newDecl, find, openScope, topScope, closeScope

def compatible(xt, yt):
    print("====compatible xt: ", xt, ", yt: ", yt, ", type(xt): ", type(xt), ", type(yt): ", type(yt))
    x = xt == Int and type(yt) == Const and yt.tp == Int or \
            xt == yt or \
            type(xt) == Set == type(yt) or \
            type(xt) == Array == type(yt) and xt.length == yt.length and \
               compatible(xt.base, yt.base) or \
            type(xt) == Record == type(yt) and \
               all(compatible(xf.tp, yf.tp) for xf, yf in zip(xt.fields, yt.fields)) or \
            type(xt) == PartialArray and type(yt) == Int or \
            type(xt) == PartialArray and yt == Int
    print("====compatible end x: ", x)
    return x

FIRSTSELECTOR = {LBRAK, PERIOD}
FIRSTFACTOR = {IDENT, NUMBER, LPAREN, NOT, CARD, COMPLEMENT}
FIRSTEXPRESSION = {PLUS, MINUS, IDENT, NUMBER, LPAREN, NOT, CARD, COMPLEMENT}
FIRSTSTATEMENT = {IDENT, IF, WHILE, FOR}
FIRSTTYPE = {IDENT, LPAREN}
FIRSTDECL = {CONST, TYPE, VAR, PROCEDURE}

def selector(x):
    while SC.sym in {LBRAK, PERIOD}:
        if SC.sym == LBRAK:  #  x[y]
            print("===0===selector [")
            getSym(); y = expression()
            if type(x.tp) == Array:
                print("===0===selector Array")
                if y.tp == Int:
                    if type(y) == Const and (y.val < x.tp.lower or y.val >= x.tp.lower + x.tp.length):
                        mark('index out of bounds')
                    else: x = CG.genIndex(x, y)
                else: mark('index not integer')
            elif type(x.tp) == PartialArray:
                print("======selector x: ", x, ", y: ", y)
                if y.tp == Int:
                    lowerBound = list(x.tp.keyValDict.keys())[0]
                    upperBound = list(x.tp.keyValDict.keys())[-1]
                    setOrUpdInd = '-1'
                    if type(y) == Const and int(y.val) >= 1 and (setOrUpdInd in x.tp.keyValDict.keys()):
                        print("===1===selector setOrUpdInd")
                        x.tp.keyValDict[y.val] = (y, None)
                        x.tp.keyValDict.pop(setOrUpdInd)
                        print("===1===selector setOrUpdInd x: ", x)
                    elif type(y) == Const and (y.val not in x.tp.keyValDict.keys()):
                        mark('index out of bounds')
                    else: print("===1===selector"); x = CG.genPartialIndex(x, y); print("===2===selector x: ", x)
                else: mark('index not integer')
            else: mark('not an array or partial array')
            if SC.sym == RBRAK: getSym()
            else: mark("] expected")
        else:  #  x.f
            print("===0===selector PERIOD")
            getSym()
            if SC.sym == IDENT:
                if type(x.tp) == Record:
                    for f in x.tp.fields:
                        if f.name == SC.val:
                            x = CG.genSelect(x, f); break
                    else: mark("not a field")
                    getSym()
                else: mark("not a record")
            else: mark("identifier expected")
    return x

def factor():
    if SC.sym == IDENT:
        x = find(SC.val)
        print("====factor IDENT x: ", x)
        if type(x) == Var: x = CG.genVar(x); getSym()
        elif type(x) == Const: x = Const(x.tp, x.val); x = CG.genConst(x); getSym()
        elif type(x) == LoopVar: getSym()
        else: mark('variable or constant identifier expected')
        x = selector(x)
    elif SC.sym == NUMBER:
        print("=====factor NUMBER")
        x = Const(Int, SC.val); print("=====factor NUMBER 1 x: ", x); x = CG.genConst(x); print("=====factor NUMBER 2 x: ", x); getSym()
    elif SC.sym == LPAREN:
        getSym(); x = expression()
        if SC.sym == RPAREN: getSym()
        else: mark(') expected')
    elif SC.sym == LBRACE:
        getSym()
        if SC.sym in FIRSTEXPRESSION:
            y = expression() 
            if y.tp == Int: x = CG.genUnaryOp(SET, y)
            else: mark('not integer')
            while SC.sym == COMMA:
                getSym(); y = expression()
                if y.tp == Int: y = CG.genUnaryOp(SET, y)
                else: mark("not integer")
                x = CG.genBinaryOp(UNION, x, y)
        else: x = Const(Set(0, 32), 0); x = CG.genConst(x)
        if SC.sym == RBRACE: getSym()
        else: mark('} expected')
    elif SC.sym == NOT:
        getSym(); x = factor()
        if x.tp != Bool: mark('not boolean')
        elif type(x) == Const: x.val = 1 - x.val # constant folding
        else: x = CG.genUnaryOp(NOT, x)
    elif SC.sym in {CARD, COMPLEMENT}:
        op = SC.sym; getSym(); x = factor()
        if type(x.tp) == Set: x = CG.genUnaryOp(op, x)
        else: mark('set expected')
    else: mark('expression expected')
    return x

def term():
    x = factor()
    while SC.sym in {TIMES, DIV, MOD, INTERSECTION, AND}:
        op = SC.sym; getSym();
        if op == AND and type(x) != Const: x = CG.genUnaryOp(AND, x)
        y = factor() # x op y
        if op in {TIMES, DIV, MOD} and x.tp == Int == y.tp:
            if type(x) == Const == type(y): # constant folding
                if op == TIMES: x.val = x.val * y.val
                elif op == DIV: x.val = x.val // y.val
                elif op == MOD: x.val = x.val % y.val
            else: x = CG.genBinaryOp(op, x, y)
        elif op == INTERSECTION and type(x.tp) == Set == type(y.tp):
            x = CG.genBinaryOp(op, x, y)
        elif op == AND and x.tp == Bool == y.tp:
            if type(x) == Const: # constant folding
                if x.val: x = y # if x is true, take y, else x
            else: x = CG.genBinaryOp(AND, x, y)
        else: mark('bad type')
    return x

def simpleExpression():
    if SC.sym == PLUS:
        getSym(); x = term()
    elif SC.sym == MINUS:
        getSym(); x = term()
        if x.tp != Int: mark('bad type')
        elif type(x) == Const: x.val = - x.val # constant folding
        else: x = CG.genUnaryOp(MINUS, x)
    else: x = term()
    while SC.sym in {PLUS, MINUS, UNION, OR}:
        op = SC.sym; getSym()
        if op == OR and type(x) != Const: x = CG.genUnaryOp(OR, x)
        y = term() # x op y
        if op in {PLUS, MINUS} and x.tp == Int == y.tp:
            if type(x) == Const == type(y): # constant folding
                if op == PLUS: x.val = x.val + y.val
                elif op == MINUS: x.val = x.val - y.val
            else: x = CG.genBinaryOp(op, x, y)
        elif op == UNION and type(x.tp) == Set == type(y.tp):
            x = CG.genBinaryOp(UNION, x, y)
        elif op == OR and x.tp == Bool == y.tp:
            if type(x) == Const: # constant folding
                if not x.val: x = y # if x is false, take y, else x
            else: x = CG.genBinaryOp(OR, x, y)
        else: print(x, y); mark('bad type')
    return x

def expression():
    x = simpleExpression()
    print("=====expression x: ", x)
    while SC.sym in {EQ, NE, LT, LE, GT, GE, ELEMENT, SUBSET, SUPERSET, ARROW}:
        op = SC.sym; getSym()
        if op in (EQ, NE, LT, LE, GT, GE):
            y = simpleExpression() # x op y
            if x.tp == y.tp in (Int, Bool):
                if type(x) == Const == type(y): # constant folding
                    if op == EQ: x.val = int(x.val == y.val)
                    elif op == NE: x.val = int(x.val != y.val)
                    elif op == LT: x.val = int(x.val < y.val)
                    elif op == LE: x.val = int(x.val <= y.val)
                    elif op == GT: x.val = int(x.val > y.val)
                    elif op == GE: x.val = int(x.val >= y.val)
                    x.tp = Bool
                else: x = CG.genRelation(op, x, y)
            else: mark('bad type')
        elif (op == ELEMENT and x.tp == Int) or \
            (op in (SUBSET, SUPERSET) and type(x.tp) == Set):
            x = CG.genUnaryOp(op, x); y = simpleExpression()
            if type(y.tp) == Set: x = CG.genRelation(op, x, y)
            else: mark('set expected')
        elif (op == ARROW and x.tp == Int):
            y = simpleExpression()
            if y.tp in (Int, Bool):
                x = (x, y)
        else: mark('bad type')
    return x

def statementList():
    x = statement()
    while SC.sym == SEMICOLON:
        getSym(); y = statement(); x = CG.genSeq(x, y)
    return x

def statementBlock():
    x = statementList()
    while SC.sym in FIRSTSTATEMENT:
        if not SC.newline: mark('new line expected')
        y = statementList(); x = CG.genSeq(x, y)
    return x

def statementSuite():
    if SC.sym in FIRSTSTATEMENT: x = statementList()
    elif SC.sym == INDENT:
        getSym(); x = statementBlock()
        if SC.sym == DEDENT: getSym();
        else: mark('dedent or new line expected')
    else: mark("indented statement expected")
    return x

def statement():
    if SC.sym == IDENT: # x := y, y(...), x ← y(...)
        # type(x) == Proc, StdProc: check no result parameters needed; call, y := true, x
        # type(x) ≠ Proc, StdProc: x := selector():
        #   sym == BECOMES: assignment; call := false
        #   sym == LARROW: check result paramter match, type(y) is Proc, StdProc, 
        x = find(SC.val)
        print("==1=statement x: ", x, ", SC.val: ", SC.val)
        if type(x) in {Proc, StdProc}: # procedure call without result
            if x.res != []: mark('variable for result expected')
            getSym(); call, xs, y = True, [], x
        elif type(x) == Var: # assignment or procedure call with result
            x = find(SC.val); print("==2=statement x: ", x, ", SC.val: ", SC.val); x = CG.genVar(x); getSym()
            print("==3=statemen x: ", x)
            if SC.sym in FIRSTSELECTOR:
                print("==4=statement SC.sym: ", SC.sym, ", type(x.tp) == PartialArray: ", type(x.tp) == PartialArray)
                if type(x.tp) == PartialArray: x.tp.keyValDict['-1'] = None # To indicate to set/update key and value
                print("==4=statement 2 x: ", x)
                xs = [CG.genLeftAssign(selector(x))] # array or field update
                print("==5=statement xs: ", xs)
            else: # multiple assignment or procedure call with result
                print("==6=statement x: ", x)
                xs = [CG.genLeftAssign(x)]
                print("==7=statement xs: ", xs)
                while SC.sym == COMMA:
                    getSym();
                    if SC.sym == IDENT:
                        x = find(SC.val)
                        if x.name not in {x.name for x in xs}:
                            if type(x) == Var:
                                xs += [CG.genLeftAssign(CG.genVar(x))]; getSym()
                            else: mark('variable identifier expected')
                        else: mark('duplicate variable identifier')
                    else: mark('identifier expected')
            if SC.sym == BECOMES:
                print("==8=statement becomes")
                getSym(); call = False; rightExpr = expression();
                print("==8=statement rightExpr: ", rightExpr, ", x: ", x)
                if type(x.tp) == PartialArray: print("==8=statement if x: ", x); ys = [CG.genRightAssign(x)]; print("==8=statement if ys: ", ys) # xs := ys
                else: print("==8=statement else xs: ", xs); ys = [CG.genRightAssign(rightExpr)]; print("==8=statement else ys: ", ys) # xs := ys
                print("==9=statement becomes xs: ", xs[0], ", ys: ", ys[0], ", ys[0].tp: ", ys[0].tp)
                while SC.sym == COMMA: getSym(); ys += [CG.genRightAssign(expression())]
                print("==10=statement becomes xs: ", xs[0], ", ys: ", ys[0])
                if len(xs) == len(ys):
                    print("==11=statement becomes zip(reversed(xs)", zip(reversed(xs), ", reversed(ys)): ", reversed(ys)))
                    for x, y in zip(reversed(xs), reversed(ys)):
                        print("==11=statement 2 becomes x.tp: ", x.tp, ", y.tp: ", y.tp)
                        if compatible(x.tp, y.tp):
                            print("==12=statement x: ", x, ", y: ", y)
                            if type(x.tp) == PartialArray:
                                rightVal = rightExpr[1]
                                print("=======statement y: ", y, ", rightVal: ", rightVal)
                                for key, value in x.tp.keyValDict.items():
                                    if value[1] is None:
                                        valList = list(value)
                                        valList[1] = rightVal.tp
                                        x.tp.keyValDict[key] = tuple(valList)
                            print("==12=statement after value set x: ", x)
                            x = CG.genAssign(x, y)
                            print("==13=statement")
                        else: mark('incompatible assignment')
                    print("==14=statement")
                else: mark('unbalanced assignment')
            elif SC.sym == LARROW:
                getSym()
                if SC.sym == IDENT: y = find(SC.val); getSym(); call = True
                else: mark('procedure identifier expected')
                if type(y) in {Proc, StdProc}:
                    if len(xs) == len(y.res):
                        for x, r in zip(xs, y.res):
                            if not compatible(x.tp, r.tp): mark('incompatible call')
                    else: mark('unbalanced call')
                else: mark('procedure expected')
            else: mark(':= or ← expected')
        else: mark("variable or procedure expected")
        if call: # call y(ap) or xs ← y(ap)
            fp, ap, i = y.par, [], 0   #  list of formals, list of actuals
            if SC.sym == LPAREN: getSym()
            else: mark("'(' expected")
            if SC.sym in FIRSTEXPRESSION:
                a = expression()
                if i < len(fp):
                    print("======statement call fp: ", fp, ", a: ", a, ", y: ", y)
                    if type(a) is tuple:
                        print("======statement call a type is tuple")
                        varPartialArray, indexVal = a[0], a[1]
                        print("======statement call tuple varPartialArray: ", varPartialArray, ", indexVal: ", indexVal)
                        if compatible(fp[i].tp, indexVal.tp):
                            print("======statement call tuple if regular")
                            ap.append(CG.genActualPara(a, fp[i], i))
                        else: mark('incompatible parameter')
                    elif type(a) == LoopVar:
                        print("======statement call a type is PartialArray for for each loops")
                        print("======statement call fp[", i, "]: ", fp[i])
                        if compatible(a.tp.tp, fp[i].tp):
                            print("======statement call if partial array")
                            ap.append(CG.genActualPara(a, fp[i], i))
                        else: mark('incompatible parameter')
                    else:
                        print("======statement call a type is not tuple")
                        print("======statement call fp[", i, "]: ", fp[i])
                        if compatible(fp[i].tp, a.tp):
                            print("======statement call if")
                            ap.append(CG.genActualPara(a.tp, fp[i], i))
                        else: mark('incompatible parameter')
                    print("======statement call end")
                else: mark('extra parameter')
                i = i + 1
                while SC.sym == COMMA:
                    getSym()
                    a = expression()
                    if i < len(fp):
                        print("======statement call comma fp: ", fp, ", a: ", a, ", y: ", y)
                        if type(a) is tuple:
                            varPartialArray, indexVal = a[0], a[1]
                            print("======statement call comma tuple varPartialArray: ", varPartialArray, ", indexVal: ", indexVal)
                            if compatible(fp[i].tp, indexVal.tp):
                                print("======statement call tuple if comma")
                                ap.append(CG.genActualPara(a, fp[i], i))
                            else: mark('incompatible parameter')
                        else:
                            if compatible(fp[i].tp, a.tp):
                                ap.append(CG.genActualPara(a.tp, fp[i], i))
                            else: mark('incompatible parameter')
                    else: mark('extra parameter')
                    i = i + 1
            if SC.sym == RPAREN: getSym()
            else: mark("')' expected")
            if i < len(fp): mark('too few parameters')
            elif type(y) == StdProc:
                if y.name == 'read': x = CG.genRead(x)
                elif y.name == 'write': x = CG.genWrite(a)
                elif y.name == 'writeln': x = CG.genWriteln()
            else: x = CG.genCall(xs, y, ap)
    elif SC.sym == IF:
        getSym(); x = expression();
        if x.tp == Bool: x = CG.genThen(x)
        else: mark('boolean expected')
        if SC.sym == THEN: getSym()
        else: mark("'then' expected")
        y = statementSuite()
        if SC.sym == ELSE:
            getSym()
            y = CG.genElse(x, y)
            z = statementSuite()
            x = CG.genIfElse(x, y, z)
        else:
            x = CG.genIfThen(x, y)
    elif SC.sym == WHILE:
        getSym(); t = CG.genWhile(); x = expression()
        if x.tp == Bool: x = CG.genDo(x)
        else: mark('boolean expected')
        if SC.sym == DO: getSym()
        else: mark("'do' expected")
        y = statementSuite()
        x = CG.genWhileDo(t, x, y)
    elif SC.sym == FOR:
        print("==16=statement FOR start")
        getSym(); x = typ()
        print("==17=statement FOR end")
    else: mark('statement expected')
    print("==18=statement x: ", x)
    return x

def typ():
    if SC.sym == IDENT:
        ident = SC.val; getSym()
        if SC.sym == IN:
            randomNumber = 0
            print("===typ if IN SC.val: ", SC.val, ", ident: ", ident)
            getSym()
            print("===typ if 2 IN SC.val: ", SC.val)
            partialArray = find(SC.val)
            print("===typ if 3 IN partialArray: ", partialArray)
            t = CG.genFor(ident, partialArray, randomNumber)
            getSym()
            if SC.sym == DO: getSym()
            else: mark("'do' expected")
            print("===typ if 4 DO sym: ", SC.sym)
            x = LoopVar(ident, partialArray)
            print("===typ if 6 IN x: ", x)
            newDecl(ident, x)
            print("===typ if 7 IN")
            y = statementSuite()
            print("===typ if 8 DO y: ", y)
            CG.genForEnd(ident, partialArray, randomNumber)
            print("===typ if 9 DO end")
        else:
            print("===typ else IN SC.val: ", SC.val, ", ident: ", ident)
            x = find(ident)
            if type(x) == Type: x = Type(x.val)
            else: mark('type identifier expected')
    elif SC.sym == LBRAK:
        getSym(); x = expression()
        arraySym = SC.sym
        keyValDict, y = {}, 0
        if arraySym == DOTDOT:
            getSym(); y = expression()
        else:
            print("=======2=======x: ", x[0], ", y", x[1], ", x[0].val: ", x[0].val)
            if type(x[0]) != Const or x[0].val < 0: mark('bad lower bound')
            keyValDict[x[0].val] = x
            print("=======2=======x:", x)
            while SC.sym == COMMA:
                print("=======3=======")
                getSym(); x = expression()
                print("=======3=======x: ", x[0], ", y", x[1], ", x[0].val: ", x[0].val)
                if type(x[0]) != Const or x[0].val < 0: mark('bad lower bound')
                keyValDict[x[0].val] = x
                print("=======3=======keyValDict: ", keyValDict)
        if SC.sym == RBRAK: getSym()
        else: mark("']' expected")
        if SC.sym == RARROW: getSym()
        else: mark("'→' expected")
        z = typ().val;
        if arraySym == DOTDOT:
            if type(x) != Const or x.val < 0: mark('bad lower bound')
            elif type(y) != Const or y.val < x.val: mark('bad upper bound')
            else: x = Type(CG.genArray(Array(z, x.val, y.val - x.val + 1)))
        else:
            print("=======4=======keyValDict: ", keyValDict)
            x = Type(CG.genPartialArray(PartialArray(z, list(keyValDict.keys())[-1], keyValDict)))
            print("=======5=======x:", x)
    elif SC.sym == LPAREN:
        getSym(); openScope(); typedIds()
        if SC.sym == RPAREN: getSym()
        else: mark("')' expected")
        r = topScope(); closeScope()
        x = Type(CG.genRec(Record(r)))
    elif SC.sym == SET:
        getSym();
        if SC.sym == LBRAK: getSym()
        else: mark ("'[' expected")
        x = expression()
        if SC.sym == DOTDOT: getSym()
        else: mark("'..' expected")
        y = expression()
        if SC.sym == RBRAK: getSym()
        else: mark("']' expected")
        if type(x) != Const: mark('bad lower bound')
        elif type(y) != Const or y.val < x.val: mark('bad upper bound')
        else: x = Type(CG.genSet(Set(x.val, y.val - x.val + 1)))
    else: mark('type expected')
    return x

def typedIds():
    if SC.sym == IDENT: tid = [SC.val]; getSym()
    else: mark("identifier expected")
    while SC.sym == COMMA:
        getSym()
        if SC.sym == IDENT: tid.append(SC.val); getSym()
        else: mark('identifier expected')
    if SC.sym == COLON: getSym()
    else: mark("':' expected")
    tp = typ().val
    print("====typedIds tid: ", tid)
    for i in tid:
        newDeclVar = None
        if isinstance(tp, PartialArray): newDeclVar = Var(copy.deepcopy(tp)); CG.genPartialIndexes(newDeclVar)
        else: newDeclVar = Var(tp)
        newDecl(i, newDeclVar)
    while SC.sym == COMMA:
        getSym()
        if SC.sym == IDENT: tid = [SC.val]; getSym()
        else: mark("identifier expected")
        while SC.sym == COMMA:
            getSym()
            if SC.sym == IDENT: tid.append(SC.val); getSym()
            else: mark('identifier expected')
        if SC.sym == COLON: getSym()
        else: mark("':' expected")
        tp = typ().val
        for i in tid:
            newDeclVar = None
            if isinstance(tp, PartialArray): newDeclVar = Var(copy.deepcopy(tp)); CG.genPartialIndexes(newDeclVar)
            else: newDeclVar = Var(tp)
            newDecl(i, newDeclVar)

def declarations(allocVar):
    print("========declarations 1")
    start = len(topScope())
    while SC.sym == CONST:
        getSym()
        if SC.sym == IDENT: ident = SC.val; getSym()
        else: mark("constant name expected")
        if SC.sym == EQ: getSym()
        else: mark("= expected")
        x = expression()
        if type(x) == Const: newDecl(ident, x)
        else: mark('expression not constant')
    print("========declarations 2")
    while SC.sym == TYPE:
        getSym()
        if SC.sym == IDENT: ident = SC.val; getSym()
        else: mark("type name expected")
        if SC.sym == EQ: getSym()
        else: mark("= expected")
        x = typ(); newDecl(ident, x)  #  x is of type ST.Type
    print("========declarations 3")
    # start = len(topScope())
    # var = allocVar(topScope(), start)
    print("========declarations 4")
    while SC.sym == VAR:
        getSym(); typedIds()
    var = allocVar(topScope(), start)
    print("========declarations 5")
    while SC.sym == PROCEDURE:
        getSym()
        if SC.sym == LPAREN:
            getSym()
            if SC.sym == IDENT: r = SC.val; getSym()
            else: mark("identifier expected")
            if SC.sym == COLON: getSym()
            else: mark("':' expected")
            tp = typ().val
            if SC.sym == RPAREN: getSym()
            else: mark(") expected")
        else: r = None
        if SC.sym == IDENT: ident = SC.val; getSym()
        else: mark("procedure name expected")
        newDecl(ident, Proc([], [])) #  entered without parameters
        sc = topScope(); openScope() # new scope for parameters and body
        if r: newDecl(r, Var(tp))
        if SC.sym == LPAREN: getSym()
        else: mark("( expected")
        if SC.sym == IDENT: typedIds()
        fp = topScope()
        if SC.sym == RPAREN: getSym()
        else: mark(") expected")
        d = len(fp)
        if SC.sym == RARROW:
            getSym()
            if SC.sym == LPAREN: getSym()
            else: mark('( expected')
            typedIds()
            if SC.sym == RPAREN: getSym()
            else: mark(') expected')
        sc[-1].par, sc[-1].res = fp[:d], fp[d:] #  procedure parameters updated
        para = CG.genProcStart(ident, fp[:d], fp[d:])
        body(ident, para); closeScope() #  scope for parameters and body closed
    print("========declarations 6")
    return var

def body(ident, para):
    print("========body 1 ident: ", ident, ", para: ", para)
    if SC.sym == INDENT: getSym()
    else: mark('indent expected')
    start = len(topScope())
    print("========body 2")
    local = declarations(CG.genLocalVars)
    print("========body 3")
    CG.genProcEntry(ident, para, local)
    print("========body 4, SC.sym: ", str(SC.sym))
    if SC.sym in FIRSTSTATEMENT: x = statementBlock()
    elif SC.sym == INDENT:
        print("========body 6")
        getSym(); x = statementBlock()
        print("========body 7")
        if SC.sym == DEDENT: getSym()
        else: mark('dedent or new line expected')
        print("========body 8")
    else: mark('statement expected')
    CG.genProcExit(x, para, local)
    print("========body 9 SC.sym: ", SC.sym)
    if SC.sym == DEDENT: getSym()
    else: mark('dedent or new line expected')
    print("========body 10")
    return x

def program():
    newDecl('boolean', Type(CG.genBool(Bool)))
    newDecl('integer', Type(CG.genInt(Int)))
    newDecl('true', Const(Bool, 1))
    newDecl('false', Const(Bool, 0))
    newDecl('read', StdProc([], [Var(Int)]))
    newDecl('write', StdProc([Var(Int)], []))
    newDecl('writeln', StdProc([], []))
    print("=====program 1")
    CG.genProgStart()
    print("=====program 2")
    declarations(CG.genGlobalVars)
    print("=====program 3")
    if SC.sym == PROGRAM: getSym()
    else: mark("'program' expected")
    print("=====program 4")
    ident = SC.val
    if SC.sym == IDENT: getSym()
    else: mark('program name expected')
    print("=====program 5")
    openScope(); print("=====program 5 1"); CG.genProgEntry(ident); print("=====program 5 2"); x = body(ident, 0); print("=====program 5 3");
    print("=====program 6")
    closeScope(); x = CG.genProgExit(x)
    print("=====program 7")
    return x

def compileString(src, dstfn = None, target = 'wat'):
    global CG
    if target == 'wat': import CGwat as CG
    elif target == 'mips': import CGmips as CG
    elif target == 'ast': import CGast as CG
    else: print('unknown target'); return
    try:
        SC.init(src); ST.init(); p = program()
        if dstfn == None: return str(p)
        else:
            with open(dstfn, 'w') as f: f.write(p)
    except Exception as msg:
        raise Exception(str(msg))

def compileFile(srcfn, target = 'wat'):
    if srcfn.endswith('.p'):
        with open(srcfn, 'r') as f: src = f.read()
        dstfn = srcfn[:-2] + '.s'
        compileString(src, dstfn, target)
    else: print("'.p' file extension expected")